#### Prequisites

In [1]:
%pip install -qU openai langchain langchain-openai langchain-community langchain-experimental pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.

In [2]:
import json
import pandas as pd
from io import StringIO
from openai import OpenAI
from IPython.display import display
from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

In [ ]:
OPENAI_API_KEY='OPENAI_API_KEY'
DOCUMENT_ID = 'DOCUMENT_ID'
SPREADSHEETS="https://docs.google.com/spreadsheets/d/{DOCUMENT_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}"

#### OpenAI API test

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)
completion = client.chat.completions.create(model="gpt-4o", messages=[{"role": "user", "content": "Jay Mataji"}])
print(completion.choices[0].message.content)

"Jay Mataji" is a phrase used to honor and invoke the blessings of a Hindu goddess, often Durga or Amba, who is revered as a mother figure. This expression is common in parts of India, particularly among Gujarati-speaking communities, and is used in various contexts, such as during religious rituals, festivals like Navratri, or as a form of greeting and solidarity among devotees. If you have any specific questions or would like more information about traditions involving Mataji, feel free to ask!


#### Read Data

In [6]:
def read_data_frame(sheet_name):
    return  pd.read_csv(SPREADSHEETS.format(DOCUMENT_ID=DOCUMENT_ID, SHEET_NAME=sheet_name))

def trim_data_frame(df, keep_rows):
    return df.iloc[keep_rows]

In [7]:
_products_df = read_data_frame('products')
_emails_df = read_data_frame('emails')

products_df = trim_data_frame(_products_df, [8, 9, 10])
emails_df = trim_data_frame(_emails_df, [6, 7, 8])

In [8]:
display(products_df)
display(emails_df)

,product_id,name,category,description,stock,seasons,price
8,BKR0123,Bucket Hat,Accessories,Protect your face from the sun in style with o...,3,"Spring, Summer",39.99
9,CBY6789,Corduroy Bucket Hat,Accessories,Keep it casual and cool with our corduroy buck...,3,"Fall, Winter",28.00
10,CLF2109,Cable Knit Beanie,Accessories,Bundle up in our cable knit beanie. Knitted fr...,2,Winter,16.00


,email_id,subject,message
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl..."
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile..."
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr..."


#### Task 1. Classify emails

In [ ]:
# Problem: Classify emails
# Tasks: Classify each email as either a "product inquiry" or an "order request". Ensure that the classification accurately reflects the intent of the email.
# Solution: Populate the "email-classification.csv" sheet with columns: email ID, category.

In [9]:
MOCK = True
if MOCK:
    mock_data_classified_email = """email ID,category
    E001, order request
    E002, product inquiry
    E003, product inquiry
    E004, order request
    E005, product inquiry
    E006, product inquiry
    E007, order request
    E008, order request
    E009, product inquiry
    E010, order request
    E011, product inquiry
    E012, product inquiry
    E013, product inquiry
    E014, order request
    E015, product inquiry
    E016, product inquiry
    E017, order request
    E018, order request
    E019, order request
    E020, product inquiry
    E021, product inquiry"""

    def classify_emails(emails_df):
        email_classification_df = pd.read_csv(StringIO(mock_data_classified_email))
        return email_classification_df
else:
    def classify_emails(emails_df):
        classifications = []
        for index, row in emails_df.iterrows():
            email_content = f"Subject: {row['subject']}\nBody: {row['message']}"
            try:
                response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "system", "content": "You are an AI model trained to classify emails."},
                        {"role": "user", "content": f"Classify the following email as 'product inquiry' or 'order request', Only respond with the category name:\n\n{email_content}"}
                    ]
                )
                category = response.choices[0].message.content
                classifications.append({'email ID': row['email_id'], 'category': str(category).lower()})
            except Exception as e:
                print(f"Error processing email ID {row['email_id']}: {e}")
                classifications.append({'email ID': row['email_id'], 'category': 'error'})

        email_classification_df = pd.DataFrame(classifications)
        return email_classification_df

In [10]:
email_classification_df = classify_emails(emails_df)
display(email_classification_df.head())

,email ID,category
0,E001,order request
1,E002,product inquiry
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry


In [11]:
email_classification_df.to_csv("email-classification.csv", index=False)

#### Task 2. Process order requests

Task 2.1:

---



In [ ]:
# Task 2.1:
# [x] Process order requests in the order they are received.
# For each request, verify product availability in stock.

# If the order can be fully fulfilled, create a new order line with the status created.
# If the order cannot be fully fulfilled, create a line with the status out of stock with quantity matching requested value.
# After placing the order, update the stock to accurately reflect the current inventory levels.
# Create a record for each product requested in the email.

# Output:
# Populate the order-status sheet with columns: email ID, product ID, quantity, status ("created", "out of stock").

In [ ]:
def init_product_langchain():
    engine = create_engine("sqlite:///products.db")
    products_df.to_sql("products", engine, index=False)
    db = SQLDatabase(engine=engine)
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        openai_api_key=OPENAI_API_KEY,
        openai_api_base=BASE_URL
    )
    return create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

In [ ]:
product_agent = init_product_langchain()

In [ ]:
def normalize_response(data):
    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict) and "id" in item:
                item["id"] = item["id"].replace(" ", "")
    elif isinstance(data, dict):
        if "id" in data:
            data["id"] = data["id"].replace(" ", "")
        data = [data]
    else:
        raise TypeError("Response data must be a list or a dictionary")
    return data

In [ ]:
def parse_agent_response(agent_response):
    output_string = agent_response['output'].replace("```json","").replace("```","")
    try:
        product_data = json.loads(output_string)
    except:
        product_data = output_string
    return product_data

In [ ]:
def extract_product_information(product_query, product_agent):
    instruction = "please output in dict format which can be parse directly from string to dict"
    agent_response = product_agent.invoke({"input": f"{product_query}, {instruction}"})

In [ ]:
def get_llm_response(use_langchain):
    if use_langchain:
        llm_response = extract_product_information(email_content, product_agent)
    else:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI model trained to parse the product details from email."},
                {"role": "user", "content": f"Parse the product details such as id,name and quantity from email in json format (if value not found mention NONE) (if all quantity is requested mention ALL):\n\n{email_content}"}
                ])
        llm_response = response.choices[0].message.content
    return llm_response

In [ ]:
def extract_order_details(email_content):
    order_details = []
    llm_response = get_llm_response(email_content)
    if any(order["id"] == "NONE" or order["quantity"] == "NONE" for order in llm_response):
        llm_response = get_llm_response(email_content, use_langchain=True)
        if any(order["id"] == "NONE" or order["quantity"] == "NONE" for order in llm_response):
            order_details = []
        else:
            order_details = normalize_response(llm_response)
    else:
        order_details = normalize_response(llm_response)

    return order_details

In [ ]:
def get_order_details(email_content):
    order_details = extract_order_details(email_content)
    return order_details

In [ ]:
def get_email_content(email_id):
    email_data = email_df[email_df["email_id"] == email_id]
    email_subject = email_data["subject"].values[0]
    email_message = email_data["message"].values[0]
    email_content = {"email_subject": email_subject, "email_message": email_message}
    return email_content

In [ ]:
def order_request(email_id):
    print(f"Processing order request for email ID: {email_id}")
    email_content = get_email_content(email_id)
    email_order_details = get_order_details(email_content)

In [ ]:
def product_inquiry(email_id):
    print(f"Processing product inquiry for email ID: {email_id}")

In [ ]:
def process_order_requests():
    for index, row in email_classification_df.iterrows():
        email_id = row["email ID"].strip()
        email_category = row["category"].strip()

        if email_category == "order request":
            order_request(email_id)
        if email_category == "product inquiry":
            product_inquiry(email_id)

        print("-" * 100)

Processing order request for email ID: E001
----------------------------------------------------------------------------------------------------
Processing product inquiry for email ID: E002
----------------------------------------------------------------------------------------------------
Processing product inquiry for email ID: E003
----------------------------------------------------------------------------------------------------
Processing order request for email ID: E004
----------------------------------------------------------------------------------------------------
Processing product inquiry for email ID: E005
----------------------------------------------------------------------------------------------------
Processing product inquiry for email ID: E006
----------------------------------------------------------------------------------------------------
Processing order request for email ID: E007
------------------------------------------------------------------------------

In [ ]:
process_order_requests()

In [ ]:
# Task 2.2:
# Generate and save response emails based on order processing results.
# Depending on the order status email should inform customer that their order was processed fully or partially, or could not be fulfilled.
# If the order was successfully processed, send an email to the customer informing them that their order has been processed, including details like the product name and quantity.
# If the order could not be fulfilled due to insufficient stock, send an email explaining the situation and specifying which items are out of stock.
# Optionally, offer options such as waiting for restock or choosing alternative products. Do your best to make an email look production ready!

# Output:
# Populate the order-response sheet with columns: email ID, response.